# Gameplan

## Columns to Fill
* company 
* year
* minerals (boolean of each)
    - Lithium
    - Cobalt
    - Gold
    - Copper
    - Gallium
    - Aluminum
    - Nickel
    - Silicon
    - Germanium
* traceability scheme
    - Responsible Minerals Initiative - RMI
    - Responsible Business Alliance - RBA
    - Fair Trade
* region  of company based 
* indutry association

## Order of Operations 
* Query all SD, grab name, year throw into list
* list of minerals
* list of traceability schemes
* Loop for name + evaluate if mineral vluaes and traceablity scheme values are true, grab year 

for seraching for metals, best to start search after "appendix A"

In [1]:
key= 'c66678a9d89152e72561cc65177356370a90e7ff6aaf7354535c2c7e95c8515f'


In [2]:
import pandas as pd
import numpy as np


# to get filings
from sec_api import FullTextSearchApi
fullTextSearchApi = FullTextSearchApi(api_key=key)

#getting the actual text document 
from sec_api import RenderApi
ra = RenderApi(api_key=key)

# for looking at the html in the jupyter notebook
from IPython.display import display, HTML
display(HTML('<h1>Impressive!</h1>'))

* get SD filings from company name, date range, and formtype SD
* filter results for company name - meta data json
* parsedoc on the results and throw results into a csv

In [5]:
## create temp table 
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df

,col1,col2
0,1,3
1,2,4


In [78]:
mineralKeyword = {
    "tableMineralsList" : [">Cobalt<",">Tin<",">Lithium<", ">Gold<", ">Copper<", ">Gallium<", ">Aluminum<", ">Nickel<", ">Silicon<", ">Germanium<"],
    "mineralsList"      : ["Cobalt", "Lithium", "Gold", "Copper", "Gallium", "Aluminum", "Nickel", "Silicon", "Germanium"],
    "mineralsString"    : " \"Cobalt\" OR \"Lithium\" OR \"Gold\" OR \"Copper\" OR \"Gallium\" OR \"Aluminum\" OR \"Nickel\" OR \"Silicon\" OR \"Germanium\" ",
    "companies"         : ["Tesla","Microsoft", "Micron", "Analog Devices", "Qorvo", "Qualcomm", "Lam Research Corporation", "Motorino", "Greenwit Technologies", "AllCell Technologies", "Bird Rides", "Hyundai", "SK Innovation", "Volkswagen", "Renault", "Nissan", "Mitsubishi", "Renault-Nissan-Mitsubishi-Alliance", "General Motors", "Toyota", "Missouri Cobalt", "L-H Battery", "Ultium", "LG Energy Solution", "Samsung", "Ford",  "Vale S.A.", "Li-Cycle Holdings", "Ascend Elements", "Electra", "Panasonic", "Neutron Holdings", "Mahindra", "EVelution Energy"],
    "cik"               : ["0001318605", "0000789019", "0000723125", "0000006281", "0001604778", "0000804328", "0000707549"],
    "auditors"          : ["Responsible Minerals Initiative", "RMI", "Responsible Business Alliance", "RBA", "Fair Trade"],
    "formTypes"         : ["SD"]
}




#searchstring = "\"" + mineralKeyword["companies"][0] + "\"" + ' AND ' +mineralKeyword["mineralsString"]  # this doesn't return only tesla
params = {
  "query": mineralKeyword["companies"][0] ,
  "formTypes": mineralKeyword["formTypes"],
}
params

{'query': 'Tesla', 'formTypes': ['SD']}

In [115]:
csvData = {
  "companyName": [],
  "filingDate": [],
  "tableMineralArray": [],
  #"mineralsArray": [],
  "auditors": [],
  "filingUrl": [],
  "cik": [],
  "accessionNo" :[]
}
response = fullTextSearchApi.get_filings(params)
filings = response['filings']
metalKeywords = mineralKeyword["tableMineralsList"]
auditKeywords = mineralKeyword["auditors"]
def parseDoc(filings,csvData,metalKeywords,auditKeywords):
  for i in filings:
    print(i["companyNameLong"])
    ##if 'tesla' in i["companyNameLong"].lower(): ## Optional
    tempMetalList = []
    tempAuditorList = []
    
    test_url = i["filingUrl"]
    doc_html = ra.get_filing(test_url)

    ## parse doc for the metals
    for k in metalKeywords:
      metalresult = doc_html.lower().find(k.lower()) 
      if metalresult > 0: 
        tempMetalList.append(k.replace('>', "").replace('<', ""))

    ## parse doc for the auditors
    for a in auditKeywords:
      auditresult = doc_html.lower().find(a.lower()) 
      if auditresult > 0: 
        tempAuditorList.append(a)

    ## Add everything to the dictionary
    if len(tempMetalList) < 1: 
      csvData["tableMineralArray"].append(['no table metals found'])
    else:
      csvData["tableMineralArray"].append(tempMetalList)

    if len(tempAuditorList) < 1: 
      csvData["auditors"].append(['no auditors found'])
    else:
      csvData["auditors"].append(tempAuditorList)

    csvData["companyName"].append(i["companyNameLong"])
    csvData["filingDate"].append(i["filedAt"])
    csvData["filingUrl"].append(i["filingUrl"])
    csvData["cik"].append(i["cik"])
    csvData["accessionNo"].append(i["accessionNo"])

  #tab = pd.DataFrame(data=csvData) ## save this for the for loop through the companies 
  return csvData

#csvData

In [123]:
def searchApiCall(params,csvData,metalKeywords,auditKeywords):
  response = fullTextSearchApi.get_filings(params)
  filings = response["filings"] 
  cf = parseDoc(filings,csvData,metalKeywords,auditKeywords)
  return cf

In [122]:

params = {
  "query": mineralKeyword["companies"][0] ,
  "formTypes": mineralKeyword["formTypes"],
}
params

{'query': 'Tesla', 'formTypes': ['SD']}

In [124]:
#filings = response['filings']
metalKeywords = mineralKeyword["tableMineralsList"]
auditKeywords = mineralKeyword["auditors"]
#parseDoc(filings,csvData,metalKeywords,auditKeywords)
t = searchApiCall(params,csvData,metalKeywords,auditKeywords)

Tesla, Inc. (TSLA) (CIK 0001318605)
Tesla, Inc. (TSLA) (CIK 0001318605)
Vizio Holding Corp. (VZIO) (CIK 0001835591)


In [125]:

params = {
  "query": mineralKeyword["companies"][1] ,
  "formTypes": mineralKeyword["formTypes"],
}
params
t2 = searchApiCall(params,csvData,metalKeywords,auditKeywords)

MICROSOFT CORP (MSFT) (CIK 0000789019)
MICROSOFT CORP (MSFT) (CIK 0000789019)
Nutanix, Inc. (NTNX) (CIK 0001618732)
SIGNET JEWELERS LTD (SIG) (CIK 0000832988)
908 Devices Inc. (MASS) (CIK 0001555279)
SPX Technologies, Inc. (SPXC) (CIK 0000088205)
MATTEL INC /DE/ (MAT) (CIK 0000063276)


In [127]:
# next step build a for loop to loop through company names and then export ot csv 

,companyName,filingDate,tableMineralArray,auditors,filingUrl,cik,accessionNo
0,"Tesla, Inc. (TSLA) (CIK 0001318605)",2024-05-31,"[Tin, Gold]","[Responsible Minerals Initiative, RMI, Respons...",https://www.sec.gov/Archives/edgar/data/131860...,1318605,0001104659-24-067119
1,"Tesla, Inc. (TSLA) (CIK 0001318605)",2024-05-31,[no table metals found],[no auditors found],https://www.sec.gov/Archives/edgar/data/131860...,1318605,0001104659-24-067119
2,Vizio Holding Corp. (VZIO) (CIK 0001835591),2024-05-31,[no table metals found],"[Responsible Minerals Initiative, RMI, Respons...",https://www.sec.gov/Archives/edgar/data/183559...,1835591,0001835591-24-000046
3,"Tesla, Inc. (TSLA) (CIK 0001318605)",2024-05-31,"[Tin, Gold]","[Responsible Minerals Initiative, RMI, Respons...",https://www.sec.gov/Archives/edgar/data/131860...,1318605,0001104659-24-067119
4,"Tesla, Inc. (TSLA) (CIK 0001318605)",2024-05-31,[no table metals found],[no auditors found],https://www.sec.gov/Archives/edgar/data/131860...,1318605,0001104659-24-067119
5,Vizio Holding Corp. (VZIO) (CIK 0001835591),2024-05-31,[no table metals found],"[Responsible Minerals Initiative, RMI, Respons...",https://www.sec.gov/Archives/edgar/data/183559...,1835591,0001835591-24-000046
6,"Tesla, Inc. (TSLA) (CIK 0001318605)",2024-05-31,"[Tin, Gold]","[Responsible Minerals Initiative, RMI, Respons...",https://www.sec.gov/Archives/edgar/data/131860...,1318605,0001104659-24-067119
7,"Tesla, Inc. (TSLA) (CIK 0001318605)",2024-05-31,[no table metals found],[no auditors found],https://www.sec.gov/Archives/edgar/data/131860...,1318605,0001104659-24-067119
8,Vizio Holding Corp. (VZIO) (CIK 0001835591),2024-05-31,[no table metals found],"[Responsible Minerals Initiative, RMI, Respons...",https://www.sec.gov/Archives/edgar/data/183559...,1835591,0001835591-24-000046
9,MICROSOFT CORP (MSFT) (CIK 0000789019),2024-05-31,"[Tin, Gold]","[Responsible Minerals Initiative, RMI, RBA]",https://www.sec.gov/Archives/edgar/data/789019...,789019,0001193125-24-151324


Some additional options

In [75]:
t1 = doc_html.lower().find('>metal<') # seems like the table are broken across separate pages 
t2 = doc_html[t1:].find('</table>')
t3 = doc_html[t2:].lower().find('>metal<')
print(t1)
print(t2)
print(t3)

69247
-1
-1


In [ ]:
display(HTML(doc_html[(t1-1000):(t1+t2)]))

In [ ]:
doc_html[(doc_html.find('gold')-100):(doc_html.find('gold')+100)]

In [ ]:
test_sub = '</table>'
test_str = doc_html

In [ ]:
res=[] 
while(test_str.find(test_sub)!=-1): 
    res.append(test_str.find(test_sub)) 
    test_str=test_str.replace(test_sub,"*"*len(test_sub),1) 

In [ ]:
res

In [ ]:
display(HTML(doc_html[res[0]:res[1]]))  

In [ ]:
res
for i in range(len(res)): 
    #print(doc_html[(res[i]-1000):(res[i]+1000)])
    display(HTML(doc_html[(res[i]):(res[i+1])]))
    
    

# JUNK 

In [ ]:
from sec_api import QueryApi
qa = QueryApi(api_key=key)

In [ ]:
query = {
  "query": "formType:SD AND companyName:Apple",
  "from": "0",
  "size": "200",
  "sort": [{ "filedAt": { "order": "desc" } }]
}


In [ ]:
response = qa.get_filings(query)